In [19]:
# Dependencies
import pandas as pd
import numpy as np
import os

In [20]:
# CSV file
csvfile = "cleaned_hawaii_measurements.csv"
csvfile2 = "cleaned_hawaii_stations.csv"

In [21]:
# Read CSV file into a pandas DataFrame
df = pd.read_csv(csvfile, dtype=object)
df2 = pd.read_csv(csvfile2, dtype=object)

In [22]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [23]:
# Create an engine to a database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [24]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [25]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# specify types for each column

Base = declarative_base()

class Measurement(Base):
    
    __tablename__ = 'measurement'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.station}"

In [26]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# specify types for each column


class Station(Base):

    __tablename__ = 'station'

    station = Column(String, primary_key=True)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.station}"

In [27]:
# Use `create_all` to create the demographics table in the database
Base.metadata.create_all(engine)

In [28]:
# Load the cleaned csv file into a pandas dataframe
df_of_data_to_insert = pd.read_csv(csvfile)
df_of_data_to_insert2 = pd.read_csv(csvfile2)

In [29]:
# Use Orient='records' to create a list of data to write
data = df_of_data_to_insert.to_dict(orient='records')
data[0]

{'date': '2010-01-01', 'prcp': 0.08, 'station': 'USC00519397', 'tobs': 65}

In [30]:
data2 = df_of_data_to_insert2.to_dict(orient='records')
data2[0]

{'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'station': 'USC00519397'}

In [31]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [32]:
# Save the reference to the `hawaii` table as a variable called `table`
table = sqlalchemy.Table('measurement', metadata, autoload=True)
table2 = sqlalchemy.Table('station', metadata, autoload=True)

In [33]:
# Use `table.delete()` to remove any existing data.
conn.execute(table.delete())
conn.execute(table2.delete())

In [34]:
# Use `table.insert()` to insert the data into the table
conn.execute(table.insert(), data)
conn.execute(table2.insert(), data2)

In [35]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]

In [36]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from station limit 5").fetchall()

[('USC00519397', None, 21.2716, -157.8168, 3.0),
 ('USC00513117', None, 21.4234, -157.8015, 14.6),
 ('USC00514830', None, 21.5213, -157.8374, 7.0),
 ('USC00517948', None, 21.3934, -157.9751, 11.9),
 ('USC00518838', None, 21.4992, -158.0111, 306.6)]